In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import evaluate
import gensim
import transformers
import nltk
import torch_geometric
import json
from xml.dom import minidom
from tqdm import tqdm

from modeling import testModel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

c:\Users\wangz\anaconda3\envs\sp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [3]:
from os import listdir
from os.path import isfile, join

mypath = "pants_straight_sides_1000"

onlyfiles = [join(mypath,f, f + "_pattern.svg") for f in listdir(mypath) if not isfile(join(mypath, f))]


In [4]:
def read_svg_string(svg_strings):
    edge_list = []
    for i in svg_strings:
        tmp = i.split()
        prev_x = float(tmp[1])
        prev_y = float(tmp[2])
        j = 3
        while j < len(tmp):
            edge = [prev_x, prev_y]
            if tmp[j] == 'z':
                break

            if tmp[j] == 'L':
                edge.append(float(tmp[j+1]))
                edge.append(float(tmp[j+2]))
                prev_x = float(tmp[j+1])
                prev_y = float(tmp[j+2])
                edge.append((edge[0] + edge[2]) / 2)
                edge.append((edge[1] + edge[3]) / 2)
                j = j + 3
                edge_list.append(edge)
                continue
            if tmp[j] == 'Q':
                prev_x = float(tmp[j+3])
                prev_y = float(tmp[j+4])
                edge.append(float(tmp[j+3]))
                edge.append(float(tmp[j+4]))
                edge.append(float(tmp[j+1]))
                edge.append(float(tmp[j+2]))
                j = j + 5
                edge_list.append(edge)
                continue
    return edge_list
            


In [5]:
edge_index = []
for i in range(20):
    base = i // 5 * 5
    edge_index.append([i, base+ (i + 1)%5])

In [8]:
def convert_to_4(data):
    start_p = np.array([data[0], data[1]])
    end_p = np.array([data[2], data[3]])
    center_p = np.array([data[4], data[5]])

    line = end_p - start_p
    p_line = np.empty_like(line)
    p_line[0] = - line[1]
    p_line[1] = line[0]

    v = center_p - start_p
    cx = np.dot(line, v) / np.linalg.norm(line)
    cy = np.dot(p_line, v) / np.linalg.norm(p_line)

    return [line[0], line[1], cx, cy]

In [16]:
def get_data_from_spec(file_path):
    doc = minidom.parse(file_path)  # parseString also exists
    path_strings = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
    res = read_svg_string(path_strings)
    res = [convert_to_4(i) for i in res]
    return res

In [7]:
edge_index = torch.tensor(edge_index).to(device).permute(1,0)
edge_index.size()

torch.Size([2, 20])

In [17]:
all_data = [get_data_from_spec(i) for i in onlyfiles]
train_data = all_data[:900]
test_data = all_data[900:]

In [23]:
lr = 1e-3
num_epochs = 50

model = testModel(4).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [24]:
#training

model.train()

for epoch in tqdm(range(num_epochs), leave=False, position=0):
    running_loss = 0
    epoch_loss = 0
        
    for data in tqdm(train_data, leave=False, position=1):
        
        input_data = torch.tensor(data).to(torch.float).to(device)
            
        # Forward pass
        out = model(input_data, edge_index)
        #print(decoder_outputs.size())
            
        

        # TODO: Compute loss
        loss = criterion(out, input_data)
        epoch_loss += loss.item()
            
            
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
            
        running_loss += loss.item()
        

    print(f'Epoch {epoch + 1} | Train Loss: {(epoch_loss / len(train_data)):.4f}')
    #eval
    eval_loss = 0
    for data in  test_data:
        input_data = torch.tensor(data).to(torch.float).to(device)
            
        # Forward pass
        out = model(input_data, edge_index)
        #print(decoder_outputs.size())
            
        

        # TODO: Compute loss
        loss = criterion(out, input_data)
        eval_loss += loss.item()
    print(f'Epoch {epoch + 1} | Eval Loss: {(eval_loss / len(test_data)):.4f}')
        
        

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 2512.8600


  2%|▏         | 1/50 [00:10<08:31, 10.45s/it]

Epoch 1 | Eval Loss: 496.9136


Epoch 2 | Train Loss: 229.5903


...

Epoch 49 | Train Loss: 2.4863


 98%|█████████▊| 49/50 [08:14<00:08,  8.81s/it]

Epoch 49 | Eval Loss: 2.5497


Epoch 50 | Train Loss: 2.6394


Epoch 50 | Eval Loss: 3.7833


In [25]:
a = get_data_from_spec(onlyfiles[-1])
a= torch.tensor(a).to(torch.float).to(device)

out = model(a, edge_index)
print(a)
print(out)

tensor([[ 123.0000,    0.0000,   61.5000,    0.0000],
        [ -18.0000, -192.0000,  173.5577,   19.2842],
        [ -75.0000,    0.0000,   37.5000,    0.0000],
        [ -30.0000,   90.0000,   71.1512,  -37.9473],
        [   0.0000,  102.0000,    5.1000,  -10.2000],
        [  93.0000,    0.0000,   46.5000,    0.0000],
        [   0.0000, -102.0000,   96.9000,  -10.2000],
        [ -30.0000,  -90.0000,   23.7171,  -33.2039],
        [ -45.0000,    0.0000,   22.5000,    0.0000],
        [ -18.0000,  192.0000,   19.2842,   19.2842],
        [ 123.0000,    0.0000,   61.5000,    0.0000],
        [   0.0000, -102.0000,   96.9000,  -10.2000],
        [ -30.0000,  -90.0000,   23.7171,  -37.9473],
        [ -75.0000,    0.0000,   37.5000,    0.0000],
        [ -18.0000,  192.0000,   19.2842,   19.2842],
        [  93.0000,    0.0000,   46.5000,    0.0000],
        [ -18.0000, -192.0000,  173.5577,   19.2842],
        [ -45.0000,    0.0000,   22.5000,    0.0000],
        [ -30.0000,   90.000